In [1]:
from scrape import *

matterhorn_URL = "https://canvas.harvard.edu/courses/69559/external_tools/22940"
panopto_URL = "https://canvas.harvard.edu/courses/69902/external_tools/61579"

In [31]:
from luigi import ExternalTask, Parameter, Task
from luigi.local_target import LocalTarget
import luigi

import pickle


class DownloadMetadata(Task):
    master_URL = Parameter()
    
#     def requires():
#         pass

    def output(self):
        return LocalTarget(os.path.join(VIDEO_PATH, 'data.plk'), format=luigi.format.Nop)
    
    def run(self):
        # do setup
        driver = setup_and_login()

        # get sources
        player_psource, player_type = get_player_psource(driver, self.master_URL)

        # get video dict
        video_dict = extract_video_links(player_psource, player=player_type)

        open_video_links(driver, video_dict, player=player_type, URL=self.master_URL) # pass in original url

        all_video_urls = get_video_urls(LOG_PATH)

        title_to_urls = get_title_to_urls(video_dict, all_video_urls, player=player_type)

        if player_type == 'matterhorn':
            title_to_urls = get_title_to_urls2(title_to_urls)
        
        data = {'title_to_urls': title_to_urls, 'player_type': player_type}
        
        with self.output().open('w') as f:
            pickle.dump(data, f)

In [33]:
from luigi import build

build([DownloadMetadata(master_URL=matterhorn_URL)], local_scheduler=True)

DEBUG: Checking if DownloadMetadata(master_URL=https://canvas.harvard.edu/courses/69559/external_tools/22940) is complete
INFO: Informed scheduler that task   DownloadMetadata_https___canvas_h_885a7e0fc5   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=561121470, workers=1, host=ET-RB-2019, username=Elliot Trilling, pid=2496) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 complete ones were encountered:
    - 1 DownloadMetadata(master_URL=https://canvas.harvard.edu/courses/69559/external_tools/22940)

Did not run any tasks
This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



True

In [ ]:
# class DownloadAllVideos(Task):
#     def requires():
#         return DownloadMetadata()
    
#     def output():
#         with open('data.plk', 'rb') as f:
#             data = pickle.load(f)
#         title_to_urls = data['title_to_urls']
#         player_type = data['player_type']
        
#         download_tasks = []

#         for title, urls in title_to_urls.items():
#             for url_num in range(len(urls)):
#                 full_title = title + ' - ' + str(url_num) + ".mp4"

#                 task = DownloadVideo(fname=full_title, download_video_params=(urls[url_num], player=player_type,
#                                                                               video_name=full_title, max_time=10)) # TODO: update time
#                 download_tasks.append(task)

#         return download_tasks
    
#     def run():
#         # I don't think we need to run anything
#         pass


# class DownloadVideo(Task):
#     fname = Paramater()
#     download_video_params = Paramater()
    
#     def requires():
#         nothing
    
#     def output():
#         LocalTarget(fname)
    
#     def run():
#         download_video(download_video_params)